<a href="https://colab.research.google.com/github/Surya-teja-888/IPL-Dataset-Player-price-prediction/blob/master/RainPrediction_using_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
upload = files.upload()

Saving Australia_rainfall.csv to Australia_rainfall.csv


In [2]:
# Importing the Modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras import layers
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import datetime
import warnings
warnings.filterwarnings("ignore")

Loading Data

In [4]:
data = pd.read_csv("Australia_rainfall.csv")
data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,01-12-2008,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,02-12-2008,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,03-12-2008,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,04-12-2008,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,05-12-2008,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


Data Preprocessing

In [5]:
# Checking for NaN values
data.isnull().sum()

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64

In [6]:
# Almost Every column in the data contains NaN values
# We need to replace the values with other values like mean, mode
# For Numerical data we can replace NaN values with mean()
# For Categorical data we can replace NaN values with mode()

In [7]:
# taking categorical_columns from the data
categorical_columns = (data.dtypes == "object")
categorical_list = list(categorical_columns[categorical_columns].index)
print(categorical_list)

['Date', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow']


In [8]:
# Replacing NaN values with mode()
for i in categorical_list:
    data[i].fillna(data[i].mode()[0],inplace = True)

In [9]:
# We replaced categorical columns with mode()
# Now it's time to replace Numerical columns with mean() of the Numerical columns
# taking numerical_data from the data
numerical_columns = (data.dtypes == "float64")
numerical_list = list(numerical_columns[numerical_columns].index)

In [10]:
# Replacing Numerical_columns with mean()
for i in numerical_list:
    data[i].fillna(data[i].mean(),inplace = True)

In [11]:
# Ckecking NaN values
data.isnull().sum()

Date             0
Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
RainToday        0
RainTomorrow     0
dtype: int64

In [16]:
new_data = data.drop(["Date","Location"],axis = 1)

Now it's time to Encode categorical features

In [19]:
x_features = new_data.columns
categorical_features = ["WindGustDir","WindDir9am","WindDir3pm","RainToday","RainTomorrow"]
encoded_data = pd.get_dummies(new_data[x_features],columns = categorical_features,drop_first = True)
encoded_data.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,WindGustDir_ENE,WindGustDir_ESE,WindGustDir_N,WindGustDir_NE,WindGustDir_NNE,WindGustDir_NNW,WindGustDir_NW,WindGustDir_S,WindGustDir_SE,WindGustDir_SSE,WindGustDir_SSW,WindGustDir_SW,WindGustDir_W,WindGustDir_WNW,WindGustDir_WSW,WindDir9am_ENE,WindDir9am_ESE,WindDir9am_N,WindDir9am_NE,WindDir9am_NNE,WindDir9am_NNW,WindDir9am_NW,WindDir9am_S,WindDir9am_SE,WindDir9am_SSE,WindDir9am_SSW,WindDir9am_SW,WindDir9am_W,WindDir9am_WNW,WindDir9am_WSW,WindDir3pm_ENE,WindDir3pm_ESE,WindDir3pm_N,WindDir3pm_NE,WindDir3pm_NNE,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW,RainToday_Yes,RainTomorrow_Yes
0,13.4,22.9,0.6,5.468232,7.611178,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.000000,4.50993,16.9,21.8,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,7.4,25.1,0.0,5.468232,7.611178,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,4.447461,4.50993,17.2,24.3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,12.9,25.7,0.0,5.468232,7.611178,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,4.447461,2.00000,21.0,23.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,9.2,28.0,0.0,5.468232,7.611178,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,4.447461,4.50993,18.1,26.5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,17.5,32.3,1.0,5.468232,7.611178,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,7.000000,8.00000,17.8,29.7,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [20]:
encoded_data.columns

Index(['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm',
       'Temp9am', 'Temp3pm', 'WindGustDir_ENE', 'WindGustDir_ESE',
       'WindGustDir_N', 'WindGustDir_NE', 'WindGustDir_NNE', 'WindGustDir_NNW',
       'WindGustDir_NW', 'WindGustDir_S', 'WindGustDir_SE', 'WindGustDir_SSE',
       'WindGustDir_SSW', 'WindGustDir_SW', 'WindGustDir_W', 'WindGustDir_WNW',
       'WindGustDir_WSW', 'WindDir9am_ENE', 'WindDir9am_ESE', 'WindDir9am_N',
       'WindDir9am_NE', 'WindDir9am_NNE', 'WindDir9am_NNW', 'WindDir9am_NW',
       'WindDir9am_S', 'WindDir9am_SE', 'WindDir9am_SSE', 'WindDir9am_SSW',
       'WindDir9am_SW', 'WindDir9am_W', 'WindDir9am_WNW', 'WindDir9am_WSW',
       'WindDir3pm_ENE', 'WindDir3pm_ESE', 'WindDir3pm_N', 'WindDir3pm_NE',
       'WindDir3pm_NNE', 'WindDir3pm_NNW', 'WindDir3pm_NW', 'WindDir3pm_S',
       'WindDir3pm_S

In [24]:
x = encoded_data.loc[:,encoded_data.columns != "RainTomorrow_Yes"]
y = encoded_data.loc[:,"RainTomorrow_Yes"]
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state = 42)

Standardization 

In [26]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [27]:
x_train.shape

(101822, 62)

In [28]:
x_test.shape

(43638, 62)

Building Neural Network

In [29]:
model = keras.Sequential([
                          layers.Dense(64,activation="relu",name="layer1"),
                          layers.Dense(64,activation="relu",name="layer2"),
                          layers.Dense(1,activation="sigmoid",name = "layer3"),
])

In [32]:
# Compiling the model
model.compile(optimizer="Adam",loss="binary_crossentropy",metrics=["accuracy"])

In [33]:
# Fitting the model
model.fit(x_train,y_train,batch_size=100,validation_split=0.3,epochs=100)

Epoch 1/100
713/713 [==============================] - 2s 2ms/step - loss: 0.4191 - accuracy: 0.8042 - val_loss: 0.3616 - val_accuracy: 0.8440
Epoch 2/100
713/713 [==============================] - 1s 2ms/step - loss: 0.3522 - accuracy: 0.8460 - val_loss: 0.3570 - val_accuracy: 0.8463
Epoch 3/100
713/713 [==============================] - 1s 2ms/step - loss: 0.3415 - accuracy: 0.8518 - val_loss: 0.3557 - val_accuracy: 0.8469
Epoch 4/100
713/713 [==============================] - 1s 2ms/step - loss: 0.3387 - accuracy: 0.8539 - val_loss: 0.3565 - val_accuracy: 0.8476
Epoch 5/100
713/713 [==============================] - 1s 2ms/step - loss: 0.3300 - accuracy: 0.8570 - val_loss: 0.3547 - val_accuracy: 0.8474
Epoch 6/100
713/713 [==============================] - 1s 2ms/step - loss: 0.3307 - accuracy: 0.8578 - val_loss: 0.3536 - val_accuracy: 0.8473
Epoch 7/100
713/713 [==============================] - 1s 2ms/step - loss: 0.3254 - accuracy: 0.8574 - val_loss: 0.3547 - val_accuracy: 0.8479

In [35]:
y_pred = model.predict(x_test)
y_pred

array([[0.8245456 ],
       [0.02396619],
       [0.67708576],
       ...,
       [0.9726943 ],
       [0.00611055],
       [0.98275644]], dtype=float32)

In [39]:
y_pred = y_pred > 0.5
y_pred

array([[ True],
       [False],
       [ True],
       ...,
       [ True],
       [False],
       [ True]])

In [40]:
y_test

100721    1
30234     0
68427     0
28624     0
31173     0
         ..
130665    1
67453     1
19257     1
13928     0
75366     0
Name: RainTomorrow_Yes, Length: 43638, dtype: uint8

In [38]:
metrics.accuracy_score(y_test,y_pred)

0.8271231495485586